In [0]:
#df1 = spark.read.format("csv").option("header", "true").load("dbfs:/FileStore/shared_uploads/waynewei.lin@mail.utoronto.ca/movies.csv")

# Additional files uploaded
# dbfs:/FileStore/shared_uploads/waynewei.lin@mail.utoronto.ca/integer.txt
# dbfs:/FileStore/shared_uploads/waynewei.lin@mail.utoronto.ca/salary_1.txt
# dbfs:/FileStore/shared_uploads/waynewei.lin@mail.utoronto.ca/shakespeare_1.txt

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *


In [0]:
# Part A
#instantiate the spark session
spark = SparkSession.builder.appName("MIE1628-Ass2").getOrCreate()

In [0]:
## Read the file to rdd
q1_rdd = spark.sparkContext.textFile("/FileStore/shared_uploads/waynewei.lin@mail.utoronto.ca/integer.txt")

# get the odd quantity
rddodd = q1_rdd.filter(lambda x: int(x)%2!=0)
oddCount = len(rddodd.collect())

# get the even quantity
rddeven = q1_rdd.filter(lambda x: int(x)%2==0)
evenCount = len(rddeven.collect())

print('Quantity for odd : %s. Quantity for even: %s .' %(oddCount, evenCount))

Quantity for odd : 496. Quantity for even: 514 .


In [0]:
## Read the file to rdd
q2_rdd = spark.sparkContext.textFile("/FileStore/shared_uploads/waynewei.lin@mail.utoronto.ca/salary_1.txt")

# let the each row become key_value type
rdddept = q2_rdd.map(lambda x: x.split(" "))
# rddresult = rdddept.reduceByKey(lambda x,y: int(x)+int(y))
# rddresult=rddresult.sortBy(lambda x:x[1], ascending=False, numPartitions=1)
# rddresult.collect()

Out[31]: [['Sales', '9136'],
 ['Research', '13391'],
 ['Developer', '22220'],
 ['QA', '31888'],
 ['Marketing', '22215'],
 ['Sales', '45567'],
 ['Research', '4023'],
 ['Developer', '7262'],
 ['QA', '5243'],
 ['Marketing', '11425'],
 ['Sales', '11956'],
 ['Research', '24149'],
 ['Developer', '18258'],
 ['QA', '22962'],
 ['Marketing', '28960'],
 ['Sales', '8766'],
 ['Research', '18343'],
 ['Developer', '13686'],
 ['QA', '27626'],
 ['Marketing', '32430'],
 ['Sales', '761'],
 ['Research', '22690'],
 ['Developer', '11703'],
 ['QA', '591'],
 ['Marketing', '15249'],
 ['Sales', '15831'],
 ['Research', '28373'],
 ['Developer', '31034'],
 ['QA', '27821'],
 ['Marketing', '25192'],
 ['Sales', '12640'],
 ['Research', '15100'],
 ['Developer', '2781'],
 ['QA', '17773'],
 ['Marketing', '9047'],
 ['Sales', '20425'],
 ['Research', '19555'],
 ['Developer', '10498'],
 ['QA', '18714'],
 ['Marketing', '1209'],
 ['Sales', '16369'],
 ['Research', '3513'],
 ['Developer', '3362'],
 ['QA', '27723'],
 ['Marketing'

In [0]:
# Apply patternmatch
import re
wordList=['Shakespeare', 'why', 'Lord', 'Library', 'GUTENBERG', 'WILLIAM', 'COLLEGE' , 'WORLD']
## Read the files to rdd
q3_rdd = spark.sparkContext.textFile("/FileStore/shared_uploads/waynewei.lin@mail.utoronto.ca/shakespeare_1.txt")
## Split the article by each notation is not word, digit and dashline.
## Apply flatMap to get  [] instead of [[]]
q3_rdd=q3_rdd.flatMap(lambda line: re.split(r'\W+', line))
q3_rdd=q3_rdd.filter(lambda x: x in wordList)
# ## Map function
q3_rdd=q3_rdd.map(lambda x: (x,int(1)))
rddresult=q3_rdd.reduceByKey(lambda x,y: x+y)
rddresult=rddresult.sortBy(lambda x:x[1], ascending=False, numPartitions=1)
rddresult.collect()

Out[33]: [('Lord', 402),
 ('WILLIAM', 128),
 ('why', 114),
 ('GUTENBERG', 100),
 ('WORLD', 98),
 ('COLLEGE', 98),
 ('Shakespeare', 22),
 ('Library', 5)]

In [0]:
## Read the files to rdd
q4_rdd = spark.sparkContext.textFile("/FileStore/shared_uploads/waynewei.lin@mail.utoronto.ca/shakespeare_1.txt")

## Split the article by each notation is not word, digit and dashline.
q4_rdd=q4_rdd.flatMap(lambda line: re.split(r'\W+', line))
## Apply flatMap to get  [] instead of [[]]
q4_rdd=q4_rdd.filter(lambda x: x !='')
## Map function
q4_rdd=q4_rdd.map(lambda x: (x,int(1)))
rddresult=q4_rdd.reduceByKey(lambda x,y: x+y)
## Sort by key's value
rddresult=rddresult.sortBy(lambda x: x[1], ascending=False, numPartitions=1)

In [0]:
## Top ten
rddresult.collect()[:10]

Out[54]: [('the', 11412),
 ('I', 9714),
 ('and', 8942),
 ('of', 7968),
 ('to', 7742),
 ('a', 5796),
 ('you', 5360),
 ('my', 4922),
 ('in', 4803),
 ('d', 4365)]

In [0]:
## Last 10
rddresult.collect()[-10:]

Out[55]: [('mourned', 1),
 ('clamors', 1),
 ('Tending', 1),
 ('Bequeathing', 1),
 ('unkindest', 1),
 ('revenged', 1),
 ('ruffle', 1),
 ('arbors', 1),
 ('forever', 1),
 ('Mischief', 1)]

In [0]:
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator, TrainValidationSplit
# Read the file to dataframe
# Ref:https://sparkbyexamples.com/pyspark/pyspark-read-csv-file-into-dataframe/
df = spark.read.format("csv").option("inferSchema", True).option("header", True).option("sep", ',').option("path", '/FileStore/shared_uploads/waynewei.lin@mail.utoronto.ca/movies.csv').load()
# checknull ref:https://stackoverflow.com/questions/44627386/how-to-find-count-of-null-and-nan-values-for-each-column-in-a-pyspark-dataframe
df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df.columns]).show()
# show the statistic result
df.describe().show()

+-------+------+------+
|movieId|rating|userId|
+-------+------+------+
|      0|     0|     0|
+-------+------+------+

+-------+------------------+------------------+------------------+
|summary|           movieId|            rating|            userId|
+-------+------------------+------------------+------------------+
|  count|              1501|              1501|              1501|
|   mean| 49.40572951365756|1.7741505662891406|14.383744170552964|
| stddev|28.937034065088994| 1.187276166124803| 8.591040424293272|
|    min|                 0|                 1|                 0|
|    max|                99|                 5|                29|
+-------+------------------+------------------+------------------+



In [0]:
# Top 20
from pyspark.sql import functions as F
df.groupBy('movieId').agg(F.mean('rating')).sort(col('avg(rating)').desc()).show(20)

+-------+------------------+
|movieId|       avg(rating)|
+-------+------------------+
|     32|2.9166666666666665|
|     90|            2.8125|
|     30|               2.5|
|     94| 2.473684210526316|
|     23| 2.466666666666667|
|     49|            2.4375|
|     29|               2.4|
|     18|               2.4|
|     52| 2.357142857142857|
|     53|              2.25|
|     62|              2.25|
|     92|2.2142857142857144|
|     46|               2.2|
|     68|2.1578947368421053|
|     87|2.1333333333333333|
|      2|2.1052631578947367|
|     69| 2.076923076923077|
|     27| 2.066666666666667|
|     88|2.0555555555555554|
|     22|              2.05|
+-------+------------------+
only showing top 20 rows



In [0]:
# Top 15 user
df.filter(df['rating']==5).groupBy('userId').count().sort(col('count').desc()).show(15)

+------+-----+
|userId|count|
+------+-----+
|    11|    8|
|    26|    6|
|    22|    6|
|    23|    6|
|    17|    5|
|    24|    5|
|     2|    5|
|    12|    4|
|    16|    4|
|     8|    4|
|    14|    4|
|    18|    4|
|    28|    3|
|     9|    2|
|    21|    2|
+------+-----+
only showing top 15 rows



In [0]:
# Q2:
# Split the dataset into train and test. Try 2 different combinations
# e.g. (60/40, 70/30, 75/25 and 80/20). (Train your model and use a
# collaborative filtering approach on 70 percent of your data and test with the
# other 30 percent and so on). Show your code and output.
tr1_df, test1_df = df.randomSplit(weights=[0.6,0.4], seed=0)
tr2_df, test2_df = df.randomSplit(weights=[0.8,0.2], seed=0)
## Recommend system
recom_system = ALS(userCol= "userId", itemCol= "movieId", ratingCol="rating", coldStartStrategy = "drop")
## Train model in different train set
s1 = recom_system.fit(tr1_df)
s2 = recom_system.fit(tr2_df)
## Get the prediction
res1 = s1.transform(test1_df)
res2 = s2.transform(test2_df)

In [0]:
res1.show(1)
res2.show(2)

+-------+------+------+----------+
|movieId|rating|userId|prediction|
+-------+------+------+----------+
|      7|     1|    28| 2.0329728|
+-------+------+------+----------+
only showing top 1 row

+-------+------+------+----------+
|movieId|rating|userId|prediction|
+-------+------+------+----------+
|      7|     1|    28| 2.1003573|
|     17|     1|    28| 0.7874142|
+-------+------+------+----------+
only showing top 2 rows



In [0]:
# Q3.
# Explain MSE, RMSE and MAE. Compare and evaluate both of
# your models with evaluation metrics (RMSE or MAE), show your code and
# print your results. Describe which one works better and why?

In [0]:
# MSE
eva_mse = RegressionEvaluator(metricName= "mse", labelCol= "rating",predictionCol= "prediction")
err1 = eva_mse.evaluate(res1)
err2 = eva_mse.evaluate(res2)

print('The mse for s1: %s. The mse for s2: %s' %(err1,err2))

The mse for s1: 1.0080610852274943. The mse for s2: 0.8094695596858505


In [0]:
# RMSE
eva_rmse = RegressionEvaluator(metricName= "rmse", labelCol= "rating",predictionCol= "prediction")
err1 = eva_rmse.evaluate(res1)
err2 = eva_rmse.evaluate(res2)

print('The rmse for s1: %s. The rmse for s2: %s' %(err1,err2))

The rmse for s1: 1.0040224525514827. The rmse for s2: 0.8997052626754223


In [0]:
# MAE
eva_mae = RegressionEvaluator(metricName= "mae", labelCol= "rating",predictionCol= "prediction")
err1 = eva_mae.evaluate(res1)
err2 = eva_mae.evaluate(res2)

print('The mae for s1: %s. The mae for s2: %s' %(err1,err2))

The mae for s1: 0.7047635377575254. The mae for s2: 0.6497222226370806


In [0]:
# Q4:
# Now tune the parameters of your algorithm to get the best set of
# parameters. Explain different parameters of the algorithm which you have
# used for tuning your algorithm. Evaluate all your models again. Show your
# code with the best values and output.

# Build the param_grid ref https://campus.datacamp.com/courses/recommendation-engines-in-pyspark/recommending-movies?ex=9
param_grid = ParamGridBuilder().addGrid(recom_system.regParam, [0.05,0.1,1]).addGrid(recom_system.rank,[5,10,20]).addGrid(recom_system.maxIter, [10,15,20]).build()
## train val split
trainvs = TrainValidationSplit(estimator=recom_system, estimatorParamMaps=param_grid, evaluator=eva_rmse)
# Train the model
model = trainvs.fit(tr2_df)
## get the model result
recom_system=model.bestModel
print('The best regularization parameter: ',recom_system._java_obj.parent().getRegParam())
print('The best ranks: ',recom_system._java_obj.parent().getRank())
print('The best maximum iterations:',recom_system._java_obj.parent().getMaxIter())
## get the final result
pred_tune = recom_system.transform(test2_df)
err_tune = eva_rmse.evaluate(pred_tune)
print('The error score is :',err_tune)

The best regularization parameter:  0.1
The best ranks:  10
The best maximum iterations: 10
The error score is : 0.8997052626754223


In [0]:
pred_tune = recom_system.transform(test1_df)
err_tune = eva_rmse.evaluate(pred_tune)
print('The error score is :',err_tune)

The error score is : 0.6905315169857084


In [0]:
# Q5
# Calculate the top 15 movie recommendations for user id 10 and
# user id 14. Show your code and output.

# Top 15 for user 10
df10 = df.filter("userId = 10")
pred10 = model.bestModel.transform(df10)
pred10.sort('prediction', ascending=False).show(15)

+-------+------+------+----------+
|movieId|rating|userId|prediction|
+-------+------+------+----------+
|      2|     4|    10| 3.1101015|
|     25|     3|    10| 2.7952857|
|     49|     3|    10|  2.713843|
|     89|     3|    10| 2.4977088|
|     42|     3|    10|  2.355872|
|      0|     3|    10| 1.9767134|
|     58|     1|    10| 1.8737819|
|     41|     2|    10| 1.8255126|
|     16|     2|    10| 1.7794192|
|     55|     2|    10| 1.6450983|
|     35|     1|    10| 1.5699247|
|      4|     3|    10| 1.5345981|
|     24|     1|    10|  1.409134|
|     13|     2|    10| 1.4072218|
|     67|     2|    10| 1.3509274|
+-------+------+------+----------+
only showing top 15 rows



In [0]:
# Top 15 for user 14
df14 = df.filter("userId = 14")
pred14 = model.bestModel.transform(df14)
pred14.sort('prediction', ascending=False).show(15)

+-------+------+------+----------+
|movieId|rating|userId|prediction|
+-------+------+------+----------+
|     29|     5|    14| 4.4985104|
|     63|     5|    14|  4.286728|
|     76|     5|    14|  4.259585|
|     25|     2|    14| 3.9678726|
|     52|     5|    14| 3.6627471|
|     72|     4|    14| 3.5141947|
|     53|     3|    14| 3.4895554|
|     96|     4|    14|   3.37187|
|     62|     4|    14|  3.204994|
|     95|     2|    14|  2.944099|
|     93|     3|    14| 2.8005564|
|     69|     3|    14| 2.6486988|
|     14|     3|    14| 2.6328382|
|     67|     3|    14|  2.514058|
|     31|     3|    14| 2.2296677|
+-------+------+------+----------+
only showing top 15 rows

